In [3]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/eb/17/fc425e1d4d86e31b2aaf0812a2ef2163763a0670d671720c7c36e8679323/streamlit-1.44.1-py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for gitpython!=3.1.19,<4,>=3.0.7 from https://files.pythonhosted.org/packages/1d/9a/4114a9057db2f1462d5c8f8390ab7383925fe1ac012eaa42402ad65c2963/GitPython-3.1.44-py3-none-any.whl.metadata
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency inform

In [20]:
%%writefile streamlit_app.py
import os
import json
from dotenv import load_dotenv
import streamlit as st

def compute_content(payload, backend_url):
    r = requests.post(
            server_url,
            json=payload.dict(),
            headers={"Content-Type": "application/json"},
            timeout=10,
        )
    return {
        "url": payload.url,
        "audience": payload.new_target_audience,
        "tone": payload.new_tone,
        "language": payload.language,
        "script": "This is a generated script based on the provided input."}

 
class ContentGeneration:
    def __init__(self, url, new_target_audience, new_tone, language):
        self.url = url
        self.new_target_audience = new_target_audience
        #self.new_tone = new_tone
        #self.language = language

 

load_dotenv()

st.title("AI-Powered Reel Content Generator")
st.write("Ingresa una url de un producto para crear el guion")

input_url = st.text_input("Ingrese la URL de un producto")
new_target_audience = st.text_input("Nuevo público objetivo")


if st.button("Generar Guion"):
    if input_url and new_target_audience and new_tone and language:
        backend_url = os.getenv("BACKEND_URL", "http://backend:8004/content_generator")
        payload = ContentGeneration(
            url=input_url,
            new_target_audience=new_target_audience,

        )
        refined_script = compute_content(payload, backend_url)

        st.header("Guion Finalizado")
        st.download_button(
            label="Descargar Guion en JSON",
            data=json.dumps(refined_script, ensure_ascii=False, indent=4),
            file_name="guion.json",
            mime="application/json"
        )
    else:
        st.warning("Por favor, completa todos los campos.")


Overwriting streamlit_app.py


In [21]:
import subprocess
import threading
import re
import sys

def run_streamlit_app():

    proc = subprocess.Popen(
        ["streamlit", "run", "streamlit_app.py"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    
 
    for line in proc.stdout:
        sys.stdout.write(line)
 
        if "Local URL:" in line:
            match = re.search(r"Local URL:\s*(http://localhost:(\d+))", line)
            if match:
                local_url = match.group(1)
                print("\nStreamlit app is running at:", local_url)
                 
                break

 
thread = threading.Thread(target=run_streamlit_app, daemon=True)
thread.start()



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8506

Streamlit app is running at: http://localhost:8506


# FastApi 

In [24]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, HttpUrl
import logging

# Define your request model.
class ContentGeneration(BaseModel):
    url: HttpUrl
    target_audience: str = "default_audience"
    tone: str = "default_tone"
    language: str = "default_language"

# Configure logging.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create the FastAPI app.
app = FastAPI(
    title="AI-Powered Text Generation with LLMs",
    description="""Create high-quality text content using advanced Large Language Models (LLMs).
                  Generate textual descriptions, narratives, and insights from images and text inputs.
                  Access the Streamlit interface at port 8501 for an interactive experience.""",
    version="0.1.0",
)

@app.get("/health")
def health_check():
    """Endpoint to check the health of the API"""
    return {"status": "ok"}

@app.post("/content_generator")
def generate_content(request: ContentGeneration):
    """Generate content based on metadata scraped from the given URL"""
    try:
        logger.info(f"Scraping metadata from URL: {request.url}")
        metadata = {"title": "Placeholder Product", "description": "Placeholder description"}
        if not metadata or not isinstance(metadata, dict):
            raise ValueError("No se pudo extraer metadata válida del producto.")
        logger.info(f"Metadata scraped: {metadata}")
        content = "Generated content placeholder based on the provided input."
        logger.info("Content generated successfully")
        return {"generated_content": content}

    except ValueError as ve:
        logger.error(f"ValueError: {ve}")
        raise HTTPException(
            status_code=400, detail={"error": "Datos inválidos", "message": str(ve)}
        )

    except Exception as e:
        logger.error(f"Error interno: {e}")
        raise HTTPException(
            status_code=500, detail={"error": "Error interno", "message": str(e)}
        )



In [25]:

import threading
import uvicorn

def run_fastapi_app():
    uvicorn.run(app, host="127.0.0.1", port=8501, log_level="info")

thread = threading.Thread(target=run_fastapi_app, daemon=True)
thread.start()

print("FastAPI app is running at http://127.0.0.1:8501")


FastAPI app is running at http://127.0.0.1:8501


INFO:     Started server process [45036]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8501): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


INFO:     127.0.0.1:61380 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:61661 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:61661 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:61661 - "GET /health HTTP/1.1" 200 OK


INFO:__main__:Scraping metadata from URL: https://example.com/
INFO:__main__:Metadata scraped: {'title': 'Placeholder Product', 'description': 'Placeholder description'}
INFO:__main__:Content generated successfully


INFO:     127.0.0.1:61752 - "POST /content_generator HTTP/1.1" 200 OK
